# Clustering Project - German U-Boats of World War II

In [3]:
import numpy as np
import pandas as pd
import sklearn.preprocessing, sklearn.cluster, sklearn.metrics
import scipy.spatial
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd
import plotly.express as px
from scipy.spatial import distance_matrix
from scipy.cluster.hierarchy import dendrogram
from sklearn.cluster import AgglomerativeClustering, DBSCAN
import matplotlib.pyplot as plt
import seaborn as sns
import requests, zipfile, io
from sklearn.cluster import DBSCAN
from sklearn.cluster import AffinityPropagation
from sklearn.cluster import OPTICS
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.metrics import silhouette_score, davies_bouldin_score, calinski_harabasz_score, make_scorer
from plotly.subplots import make_subplots
from sklearn.mixture import GaussianMixture
from sklearn.model_selection import GridSearchCV
from scipy.stats import pearsonr, spearmanr

##### 
- Zdroj dat: https://www.kaggle.com/datasets/cormac42/ww2-u-boats
- Dataset obsahuje informace o německých ponorkách z druhé světové války.
- Data jsou scrapována z wikipedia stránek.


## Preprocessing

In [5]:
# Load the data
df = pd.read_csv('uboats.csv')
df

,Name,Year,Type,Notable Commanders,Warships_sunk_n_total_loss_No,Warships_sunk_n_total_loss_Tons-n-GRT,Warships_Damaged_No,Warships_Damaged_Tons-n-GRT,Merchant_Ships_sunk_No,Merchant_Ships_sunk_GRT,...,Notes,URL,Commissioned,Patrols,Patrols_Count,Wolfpacks,Wolfpacks_Count,Flotilla,Flotilla_Count,Last_Flotilla
0,U-1,1935,IIA,Klaus Ewerth,0,0,0,0,0,0,...,Struck a mine,https://en.wikipedia.org/wiki/German_submarine...,1935-06-29,2 patrols:1st patrol:15 – 29 March 19402nd pat...,2,NaN,0,"{'U-boat School Flotilla': ['1 July 1935 ', ' ...",1,U-boat School Flotilla
1,U-2,1935,IIA,"Hans Heidtmann,Heinrich Liebe,Helmut Rosenbaum...",0,0,0,0,0,0,...,Training boat,https://en.wikipedia.org/wiki/German_submarine...,1935-07-25,2 patrols:1st patrol:15 – 29 March 19402nd pat...,2,NaN,0,"{'U-boat School Flotilla': ['1 July 1935 ', ' ...",2,21st U-boat Flotilla
2,U-3,1935,IIA,"Joachim Schepke,Otto von Bülow,Hans-Hartwig Tr...",0,0,0,0,2,"2,348",...,NaN,https://en.wikipedia.org/wiki/German_submarine...,1935-09-06,5 patrols:1st patrol:4 – 8 September 19392nd p...,5,NaN,0,"{'U-boat School Flotilla': ['1 August 1935 ', ...",2,21st U-boat Flotilla
3,U-4,1935,IIA,Heinz-Otto Schultze,1,"1,090",0,0,3,"5,133",...,NaN,https://en.wikipedia.org/wiki/German_submarine...,1935-08-17,4 patrols:1st patrol:4 – 14 September 19392nd ...,4,NaN,0,"{'U-boat School Flotilla': ['1 August 1935 ', ...",2,21st U-boat Flotilla
4,U-5,1935,IIA,Heinrich Lehmann-Willenbrock,0,0,0,0,0,0,...,Accident,https://en.wikipedia.org/wiki/German_submarine...,1935-08-31,2 patrols:1st patrol:24 August – 8 September 1...,2,NaN,0,{'U-boat School Flotilla': ['1 September 1935 ...,2,21st U-boat Flotilla
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1148,U-4707,1945,XXIII,NaN,0,0,0,0,0,0,...,NaN,https://en.wikipedia.org/wiki/German_submarine...,1945-02-20,NaN,0,NaN,0,"{'5th U-boat Flotilla': ['20 February ', ' 5 M...",1,5th U-boat Flotilla
1149,U-4709,1945,XXIII,NaN,0,0,0,0,0,0,...,NaN,https://en.wikipedia.org/wiki/German_submarine...,1945-03-03,NaN,0,NaN,0,"{'5th U-boat Flotilla': ['3 March ', ' 4 May 1...",1,5th U-boat Flotilla
1150,U-4710,1945,XXIII,Ludwig von Friedeburg,0,0,0,0,0,0,...,NaN,https://en.wikipedia.org/wiki/German_submarine...,1945-05-01,NaN,0,NaN,0,"{'5th U-boat Flotilla': ['1 ', ' 5 May 1945']}",1,5th U-boat Flotilla
1151,U-4711,1945,XXIII,NaN,0,0,0,0,0,0,...,NaN,https://en.wikipedia.org/wiki/German_submarine...,1945-03-21,NaN,0,NaN,0,"{'5th U-boat Flotilla': ['21 March ', ' 4 May ...",1,5th U-boat Flotilla


In [15]:
# print table where will be each column with its name, type, number of missing values and unique values
def print_columns_descrbition(df):
    tab = pd.DataFrame(columns=['column_name', 'type', 'missing', 'unique'])
    for col in df.columns:
        new_row = pd.DataFrame({'column_name': [col], 'type': [df[col].dtype], 'missing': [df[col].isnull().sum()], 'unique': [df[col].nunique()]})
        tab = pd.concat([tab, new_row], ignore_index=True)
    return tab

# print number of rows and columns
print(f"Table with some description of columns:\nOriginal dataset has {df.shape[0]} rows and {df.shape[1]} columns")
print_columns_descrbition(df)

Table with some description of columns:
Original dataset has 1153 rows and 26 columns


,column_name,type,missing,unique
0,Name,object,0,1153
1,Year,int64,0,11
2,Type,object,0,24
3,Notable Commanders,object,935,169
4,Warships_sunk_n_total_loss_No,int64,0,7
5,Warships_sunk_n_total_loss_Tons-n-GRT,object,0,156
6,Warships_Damaged_No,int64,0,4
7,Warships_Damaged_Tons-n-GRT,object,0,46
8,Merchant_Ships_sunk_No,int64,0,39
9,Merchant_Ships_sunk_GRT,object,0,423


In [16]:
# print fate events unique values and their counts
df['Fate_Event'].value_counts()

Fate_Event
Sunk                       650
Scuttled                   268
Surrendered                149
Missing                     45
Stricken                    15
Captured                     6
Scrapped                     5
Broken up                    4
Buried                       3
Decommissioned               1
Sold                         1
Ran aground & destroyed      1
Interned, then sold          1
Destroyed                    1
Interned                     1
Grounded                     1
Aground                      1
Name: count, dtype: int64

### Popis datasetu
- Dataset obsahuje 1153 ponerek vyrobených Německem od roku 1935 do roku 1945.
- Neobsahuje ponorky, které byly německem ukořistěny a použity německým námořnictvem.

### Popis sloupců
- **Name** - název ponorky
- **Year** - rok, kdy byla ponorka spuštěna na vodu
- **Type** - typ ponorky
- **Notable Commanders** - významní velitelé ponorky
- **Warships_sunk_n_total_loss_No** - počet potopených válečných lodí
- **Warships_sunk_n_total_loss_Tons-n-GRT** - celková tonáž potopených válečných lodí
- **Warships_Damaged_No** - počet válečných lodí, které byly poškozeny
- **Warships_Damaged_Tons-n-GRT** - celková tonáž válečných lodí, které byly poškozeny
- **Merchant_Ships_sunk_No** - počet potopených obchodních lodí
- **Merchant_Ships_sunk_GRT** - celková tonáž potopených obchodních lodí
- **Merchant_Ships_Damaged_No** - počet obchodních lodí, které byly poškozeny
- **Merchant_Ships_Damaged_GRT** - celková tonáž obchodních lodí, které byly poškozeny
- **Merchant_Ships_total_loss_No** - počet obchodních lodí, které byly ztraceny
- **Merchant_Ships_total_loss_GRT** - celková tonáž obchodních lodí, které byly ztraceny
- **Fate_Event** - osud ponorky
- **Fate_Date** - datum, kdy k osudu ponorky došlo
- **Notes** - poznámky(Např. název lodě, která ponorku potopila; způsob potopení; způsob použití; atd.)

In [ ]:
# TODO - drop columns which will not be used
# TODO - missing values
# TODO - encoding
# TODO - scaling
# TODO - slocit sunk and destroyeD?
# TODO - correlation
# TODO - outliers
# TODO - popis co jsme zatim zjistili

In [ ]:
# TODO - clustering 3 methods (kmeans...)
# - on scaled and unscaled data
# only with numerical columns 